# GNOSIS Trading System - Backtesting Notebook

This notebook provides an interactive interface for backtesting the GNOSIS multi-agent trading system.

## Features
- **4 Trading Engines**: Hedge, Liquidity, Sentiment, Elasticity
- **Composer Consensus**: Weighted voting system for trade signals
- **Real Historical Data**: Powered by Alpaca (5+ years of data)
- **Optional LSTM**: ML-enhanced predictions
- **Comprehensive Metrics**: Sharpe, Sortino, Calmar, Max Drawdown, Win Rate

## 1. Setup and Imports

In [ ]:
# Standard library
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
ROOT_DIR = Path(os.getcwd()).parent if 'notebooks' in os.getcwd() else Path(os.getcwd())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

print(f"Project root: {ROOT_DIR}")
print(f"Python version: {sys.version}")

In [ ]:
# Data analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import PercentFormatter

# Interactive widgets (optional)
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("ipywidgets not available - using static configuration")

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Imports loaded successfully!")

In [ ]:
# Import GNOSIS backtesting engine
from backtesting.ml_backtest_engine import (
    MLBacktestConfig,
    MLBacktestEngine,
    MLBacktestResults,
    BacktestTrade,
    print_results_summary,
    run_ml_backtest,
)

print("GNOSIS backtesting engine loaded!")

## 2. API Credentials

API credentials are pre-configured for:
- **Alpaca**: Paper trading account for historical market data
- **Unusual Whales**: Options flow data

Run the cell below to set the credentials.

In [ ]:
# =============================================================================
# API CREDENTIALS (Hardcoded)
# =============================================================================

# Alpaca API (Paper Trading)
os.environ['ALPACA_API_KEY'] = 'PKDGAH5CJM4G3RZ2NP5WQNH22U'
os.environ['ALPACA_SECRET_KEY'] = 'EfW43tDsmhWgvJkucKhJL3bsXmKyu5Kt1B3WxTFcuHEq'
os.environ['ALPACA_BASE_URL'] = 'https://paper-api.alpaca.markets/v2'

# Unusual Whales API
os.environ['UNUSUAL_WHALES_API_TOKEN'] = '8932cd23-72b3-4f74-9848-13f9103b9df5'

# MASSIVE.COM API
os.environ['MASSIVE_API_KEY'] = 'Jm_fqc_gtSTSXG78P67dpBpO3LX_4P6D'
# Secondary MASSIVE key (fallback)
os.environ['MASSIVE_API_KEY_SECONDARY'] = '22265906-ec01-4a42-928a-0037ccadbde3'
os.environ['MASSIVE_API_ENABLED'] = 'true'

# Verify credentials are set
print("API Credentials Configured:")
print(f"  Alpaca API Key: {os.environ['ALPACA_API_KEY'][:8]}...{os.environ['ALPACA_API_KEY'][-4:]}")
print(f"  Alpaca Base URL: {os.environ['ALPACA_BASE_URL']}")
print(f"  Unusual Whales Token: {os.environ['UNUSUAL_WHALES_API_TOKEN'][:8]}...")
print(f"  MASSIVE API Key: {os.environ['MASSIVE_API_KEY'][:8]}...{os.environ['MASSIVE_API_KEY'][-4:]}")


## 3. Backtest Configuration

Configure the backtest parameters below. The system supports:
- **Timeframes**: 1Min, 5Min, 15Min, 30Min, 1Hour, 4Hour, 1Day
- **Symbols**: Any US stock or ETF (SPY, QQQ, AAPL, TSLA, etc.)
- **Date Range**: Up to 5+ years of history

In [ ]:
# =============================================================================
# BACKTEST CONFIGURATION
# =============================================================================

# Symbol and Date Range
SYMBOL = "SPY"                    # Trading symbol
START_DATE = "2022-01-01"         # Start date (YYYY-MM-DD)
END_DATE = "2024-12-01"           # End date (YYYY-MM-DD)
TIMEFRAME = "1Day"                # 1Min, 5Min, 15Min, 30Min, 1Hour, 4Hour, 1Day

# Capital and Position Sizing
INITIAL_CAPITAL = 100_000         # Starting capital ($)
POSITION_SIZE_PCT = 0.10          # Position size as % of capital (10%)
MAX_POSITIONS = 1                 # Maximum concurrent positions

# Risk Management
STOP_LOSS_PCT = 0.02              # Stop loss (2%)
TAKE_PROFIT_PCT = 0.04            # Take profit (4%)

# Signal Thresholds
ENTRY_THRESHOLD = 0.3             # Minimum consensus for entry (-1 to 1 scale)
EXIT_THRESHOLD = 0.1              # Exit on signal reversal threshold

# Engine Weights (should sum to 1.0)
HEDGE_WEIGHT = 0.4                # Weight for Hedge engine
LIQUIDITY_WEIGHT = 0.2            # Weight for Liquidity engine
SENTIMENT_WEIGHT = 0.4            # Weight for Sentiment engine

# Cost Modeling
SLIPPAGE_BPS = 5.0                # Slippage in basis points
IMPACT_BPS = 5.0                  # Market impact in basis points

# ML Settings (optional)
USE_LSTM = False                  # Enable LSTM predictions
LSTM_MODEL_PATH = None            # Path to trained LSTM model
LSTM_WEIGHT = 0.3                 # LSTM weight in final signal

# Output Settings
TAG = f"{SYMBOL}_backtest"        # Run identifier
OUTPUT_DIR = "../runs/backtests"  # Output directory
SAVE_RESULTS = True               # Save results to disk

print("Configuration loaded!")
print(f"\nBacktest: {SYMBOL} from {START_DATE} to {END_DATE} ({TIMEFRAME})")
print(f"Capital: ${INITIAL_CAPITAL:,.0f} | Position Size: {POSITION_SIZE_PCT*100:.0f}%")
print(f"Stop Loss: {STOP_LOSS_PCT*100:.1f}% | Take Profit: {TAKE_PROFIT_PCT*100:.1f}%")

In [ ]:
# Build the configuration object
config = MLBacktestConfig(
    # Symbol and dates
    symbol=SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    timeframe=TIMEFRAME,
    
    # Capital
    initial_capital=INITIAL_CAPITAL,
    position_size_pct=POSITION_SIZE_PCT,
    max_positions=MAX_POSITIONS,
    
    # Costs
    slippage_bps=SLIPPAGE_BPS,
    impact_bps=IMPACT_BPS,
    
    # Risk management
    stop_loss_pct=STOP_LOSS_PCT,
    take_profit_pct=TAKE_PROFIT_PCT,
    
    # Signal thresholds
    entry_threshold=ENTRY_THRESHOLD,
    exit_threshold=EXIT_THRESHOLD,
    
    # Engine weights
    hedge_weight=HEDGE_WEIGHT,
    liquidity_weight=LIQUIDITY_WEIGHT,
    sentiment_weight=SENTIMENT_WEIGHT,
    
    # ML settings
    use_lstm=USE_LSTM,
    lstm_model_path=LSTM_MODEL_PATH,
    lstm_weight=LSTM_WEIGHT,
    
    # Output
    tag=TAG,
    output_dir=OUTPUT_DIR,
    save_trades=SAVE_RESULTS,
    save_equity_curve=SAVE_RESULTS,
)

print("Config object created!")

## 4. Run Backtest

Execute the backtest with the configured parameters. This will:
1. Fetch historical data from Alpaca
2. Run all 4 engines on each bar
3. Compute composer consensus
4. Execute trades based on signals
5. Calculate performance metrics

In [ ]:
# Initialize and run the backtest engine
print("=" * 60)
print("STARTING BACKTEST")
print("=" * 60)
print(f"Symbol:    {config.symbol}")
print(f"Period:    {config.start_date} to {config.end_date}")
print(f"Timeframe: {config.timeframe}")
print(f"Capital:   ${config.initial_capital:,.0f}")
print("=" * 60)
print("\nFetching data and running backtest...\n")

engine = MLBacktestEngine(config)
results = engine.run_backtest()

print("\nBacktest complete!")

In [ ]:
# Print summary
print_results_summary(results)

## 5. Results Analysis

### 5.1 Key Performance Metrics

In [ ]:
# Create metrics summary DataFrame
metrics = {
    'Metric': [
        'Total Return ($)',
        'Total Return (%)',
        'Total Trades',
        'Win Rate',
        'Profit Factor',
        'Sharpe Ratio',
        'Sortino Ratio',
        'Max Drawdown (%)',
        'Calmar Ratio',
        'Volatility (%)',
        'Avg Win ($)',
        'Avg Loss ($)',
        'Largest Win ($)',
        'Largest Loss ($)',
        'Total Costs ($)',
    ],
    'Value': [
        f"${results.total_return:,.2f}",
        f"{results.total_return_pct*100:.2f}%",
        f"{results.total_trades}",
        f"{results.win_rate*100:.1f}%",
        f"{results.profit_factor:.2f}",
        f"{results.sharpe_ratio:.2f}",
        f"{results.sortino_ratio:.2f}",
        f"{results.max_drawdown_pct*100:.2f}%",
        f"{results.calmar_ratio:.2f}",
        f"{results.volatility*100:.1f}%",
        f"${results.avg_win:,.2f}",
        f"${results.avg_loss:,.2f}",
        f"${results.largest_win:,.2f}",
        f"${results.largest_loss:,.2f}",
        f"${results.total_costs:,.2f}",
    ]
}

metrics_df = pd.DataFrame(metrics)
print("\nKEY PERFORMANCE METRICS")
print("=" * 40)
print(metrics_df.to_string(index=False))

### 5.2 Equity Curve

In [ ]:
# Convert equity curve to DataFrame
if results.equity_curve:
    equity_df = pd.DataFrame(results.equity_curve)
    equity_df['timestamp'] = pd.to_datetime(equity_df['timestamp'])
    equity_df.set_index('timestamp', inplace=True)
    
    # Calculate drawdown
    equity_df['peak'] = equity_df['equity'].expanding().max()
    equity_df['drawdown'] = (equity_df['equity'] - equity_df['peak']) / equity_df['peak']
    
    print(f"Equity curve: {len(equity_df)} data points")
    print(f"Date range: {equity_df.index.min()} to {equity_df.index.max()}")
else:
    print("No equity curve data available")

In [ ]:
# Plot equity curve and drawdown
if results.equity_curve:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [3, 1]})
    
    # Equity curve
    ax1 = axes[0]
    ax1.plot(equity_df.index, equity_df['equity'], color='blue', linewidth=1.5, label='Portfolio Value')
    ax1.axhline(y=config.initial_capital, color='gray', linestyle='--', alpha=0.7, label='Initial Capital')
    ax1.fill_between(equity_df.index, config.initial_capital, equity_df['equity'], 
                     where=(equity_df['equity'] >= config.initial_capital), 
                     color='green', alpha=0.3)
    ax1.fill_between(equity_df.index, config.initial_capital, equity_df['equity'], 
                     where=(equity_df['equity'] < config.initial_capital), 
                     color='red', alpha=0.3)
    ax1.set_title(f'{SYMBOL} Backtest Equity Curve ({START_DATE} to {END_DATE})', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
    ax1.legend(loc='upper left')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    ax1.grid(True, alpha=0.3)
    
    # Drawdown
    ax2 = axes[1]
    ax2.fill_between(equity_df.index, 0, equity_df['drawdown'], color='red', alpha=0.5)
    ax2.plot(equity_df.index, equity_df['drawdown'], color='darkred', linewidth=1)
    ax2.set_title('Drawdown', fontsize=12)
    ax2.set_ylabel('Drawdown (%)', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax2.set_ylim([min(equity_df['drawdown']) * 1.1, 0.01])
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No equity curve data to plot")

### 5.3 Trade Analysis

In [ ]:
# Convert trades to DataFrame
if results.trades:
    trades_data = []
    for t in results.trades:
        trades_data.append({
            'Entry Date': t.entry_date,
            'Exit Date': t.exit_date,
            'Direction': t.direction,
            'Entry Price': t.entry_price,
            'Exit Price': t.exit_price,
            'Position Size': t.position_size,
            'Net P&L': t.net_pnl,
            'P&L %': t.pnl_pct * 100,
            'Winner': t.is_winner,
            'Exit Reason': t.exit_reason,
            'Composite Signal': t.composite_signal,
            'Confidence': t.confidence,
        })
    
    trades_df = pd.DataFrame(trades_data)
    trades_df['Entry Date'] = pd.to_datetime(trades_df['Entry Date'])
    trades_df['Exit Date'] = pd.to_datetime(trades_df['Exit Date'])
    
    print(f"\nTOTAL TRADES: {len(trades_df)}")
    print(f"Winners: {trades_df['Winner'].sum()} | Losers: {len(trades_df) - trades_df['Winner'].sum()}")
    print("\nFirst 10 trades:")
    display(trades_df.head(10).style.format({
        'Entry Price': '${:.2f}',
        'Exit Price': '${:.2f}',
        'Position Size': '{:.2f}',
        'Net P&L': '${:.2f}',
        'P&L %': '{:.2f}%',
        'Composite Signal': '{:.3f}',
        'Confidence': '{:.2f}',
    }))
else:
    print("No trades executed")
    trades_df = pd.DataFrame()

In [ ]:
# Trade analysis visualizations
if len(trades_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. P&L Distribution
    ax1 = axes[0, 0]
    colors = ['green' if x > 0 else 'red' for x in trades_df['Net P&L']]
    ax1.bar(range(len(trades_df)), trades_df['Net P&L'], color=colors, alpha=0.7)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.set_title('P&L by Trade', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Trade #')
    ax1.set_ylabel('P&L ($)')
    ax1.grid(True, alpha=0.3)
    
    # 2. Cumulative P&L
    ax2 = axes[0, 1]
    cumulative_pnl = trades_df['Net P&L'].cumsum()
    ax2.plot(range(len(trades_df)), cumulative_pnl, color='blue', linewidth=2)
    ax2.fill_between(range(len(trades_df)), 0, cumulative_pnl, 
                     where=(cumulative_pnl >= 0), color='green', alpha=0.3)
    ax2.fill_between(range(len(trades_df)), 0, cumulative_pnl, 
                     where=(cumulative_pnl < 0), color='red', alpha=0.3)
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.set_title('Cumulative P&L', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Trade #')
    ax2.set_ylabel('Cumulative P&L ($)')
    ax2.grid(True, alpha=0.3)
    
    # 3. Win/Loss Distribution
    ax3 = axes[1, 0]
    winners = trades_df[trades_df['Winner']]['Net P&L']
    losers = trades_df[~trades_df['Winner']]['Net P&L']
    ax3.hist([winners, losers.abs()], bins=20, label=['Wins', 'Losses'], 
             color=['green', 'red'], alpha=0.7)
    ax3.set_title('Win/Loss Distribution', fontsize=12, fontweight='bold')
    ax3.set_xlabel('P&L ($)')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Exit Reasons
    ax4 = axes[1, 1]
    exit_counts = trades_df['Exit Reason'].value_counts()
    colors_exit = {'stop_loss': 'red', 'take_profit': 'green', 
                   'signal_reversal': 'orange', 'end_of_test': 'gray'}
    bars = ax4.bar(exit_counts.index, exit_counts.values, 
                   color=[colors_exit.get(x, 'blue') for x in exit_counts.index], alpha=0.7)
    ax4.set_title('Exit Reasons', fontsize=12, fontweight='bold')
    ax4.set_xlabel('Exit Reason')
    ax4.set_ylabel('Count')
    ax4.grid(True, alpha=0.3)
    
    # Add count labels on bars
    for bar, count in zip(bars, exit_counts.values):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                 str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

### 5.4 Monthly Returns Heatmap

In [ ]:
# Calculate monthly returns
if results.equity_curve:
    # Resample equity curve to monthly
    monthly_equity = equity_df['equity'].resample('ME').last()
    monthly_returns = monthly_equity.pct_change().dropna()
    
    # Create pivot table for heatmap
    monthly_returns_df = pd.DataFrame({
        'Year': monthly_returns.index.year,
        'Month': monthly_returns.index.month,
        'Return': monthly_returns.values * 100
    })
    
    pivot = monthly_returns_df.pivot(index='Year', columns='Month', values='Return')
    pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(14, 4))
    
    # Create heatmap manually
    cmap = plt.cm.RdYlGn
    vmax = max(abs(pivot.values.min()), abs(pivot.values.max())) if not pivot.empty else 10
    
    im = ax.imshow(pivot.values, cmap=cmap, aspect='auto', vmin=-vmax, vmax=vmax)
    
    # Set ticks
    ax.set_xticks(np.arange(len(pivot.columns)))
    ax.set_yticks(np.arange(len(pivot.index)))
    ax.set_xticklabels(pivot.columns)
    ax.set_yticklabels(pivot.index)
    
    # Add text annotations
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            val = pivot.iloc[i, j]
            if not np.isnan(val):
                text = ax.text(j, i, f'{val:.1f}%', ha='center', va='center', 
                              color='black' if abs(val) < vmax * 0.5 else 'white', fontsize=9)
    
    ax.set_title('Monthly Returns (%)', fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Return (%)')
    plt.tight_layout()
    plt.show()
    
    # Print annual returns
    annual_returns = equity_df['equity'].resample('YE').last().pct_change().dropna()
    print("\nANNUAL RETURNS:")
    for date, ret in annual_returns.items():
        print(f"  {date.year}: {ret*100:+.2f}%")

## 6. Signal Analysis

Analyze the consensus signals over time

In [ ]:
# Plot consensus signal over time
if results.equity_curve:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Equity with trades marked
    ax1 = axes[0]
    ax1.plot(equity_df.index, equity_df['equity'], color='blue', linewidth=1, label='Equity')
    
    # Mark trade entries and exits
    if len(trades_df) > 0:
        for _, trade in trades_df.iterrows():
            entry_idx = trade['Entry Date']
            if entry_idx in equity_df.index:
                color = 'green' if trade['Direction'] == 'long' else 'red'
                ax1.axvline(x=entry_idx, color=color, alpha=0.3, linewidth=0.5)
    
    ax1.set_title(f'{SYMBOL} Equity with Trade Markers', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    ax1.grid(True, alpha=0.3)
    
    # Consensus signal
    ax2 = axes[1]
    colors = ['green' if x > 0 else 'red' for x in equity_df['consensus']]
    ax2.bar(equity_df.index, equity_df['consensus'], color=colors, alpha=0.5, width=1)
    ax2.axhline(y=ENTRY_THRESHOLD, color='green', linestyle='--', alpha=0.7, label=f'Entry ({ENTRY_THRESHOLD})')
    ax2.axhline(y=-ENTRY_THRESHOLD, color='red', linestyle='--', alpha=0.7, label=f'Entry ({-ENTRY_THRESHOLD})')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax2.set_title('Composer Consensus Signal', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Consensus (-1 to 1)')
    ax2.set_xlabel('Date')
    ax2.set_ylim([-1.1, 1.1])
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Multi-Symbol Comparison

Compare backtest results across multiple symbols

In [ ]:
# Run backtests for multiple symbols
COMPARE_SYMBOLS = ['SPY', 'QQQ', 'IWM']  # Edit this list as needed
RUN_COMPARISON = False  # Set to True to run comparison

if RUN_COMPARISON:
    comparison_results = {}
    
    for symbol in COMPARE_SYMBOLS:
        print(f"\nRunning backtest for {symbol}...")
        try:
            sym_results = run_ml_backtest(
                symbol=symbol,
                start_date=START_DATE,
                end_date=END_DATE,
                timeframe=TIMEFRAME,
                initial_capital=INITIAL_CAPITAL,
                tag=f"{symbol}_comparison",
                position_size_pct=POSITION_SIZE_PCT,
                stop_loss_pct=STOP_LOSS_PCT,
                take_profit_pct=TAKE_PROFIT_PCT,
            )
            comparison_results[symbol] = sym_results
            print(f"  {symbol}: Return={sym_results.total_return_pct*100:.2f}%, Sharpe={sym_results.sharpe_ratio:.2f}")
        except Exception as e:
            print(f"  {symbol}: ERROR - {e}")
    
    print("\nComparison complete!")
else:
    print("Multi-symbol comparison disabled.")
    print("Set RUN_COMPARISON = True to run backtests for multiple symbols.")
    comparison_results = {}

In [ ]:
# Display comparison table
if comparison_results:
    comparison_data = []
    for symbol, res in comparison_results.items():
        comparison_data.append({
            'Symbol': symbol,
            'Return (%)': f"{res.total_return_pct*100:.2f}%",
            'Trades': res.total_trades,
            'Win Rate': f"{res.win_rate*100:.1f}%",
            'Sharpe': f"{res.sharpe_ratio:.2f}",
            'Sortino': f"{res.sortino_ratio:.2f}",
            'Max DD': f"{res.max_drawdown_pct*100:.2f}%",
            'Profit Factor': f"{res.profit_factor:.2f}",
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\nMULTI-SYMBOL COMPARISON")
    print("=" * 80)
    display(comparison_df)
    
    # Plot comparison
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for symbol, res in comparison_results.items():
        if res.equity_curve:
            eq_df = pd.DataFrame(res.equity_curve)
            eq_df['timestamp'] = pd.to_datetime(eq_df['timestamp'])
            eq_df.set_index('timestamp', inplace=True)
            # Normalize to percentage return
            normalized = (eq_df['equity'] / eq_df['equity'].iloc[0] - 1) * 100
            ax.plot(eq_df.index, normalized, label=symbol, linewidth=1.5)
    
    ax.set_title('Multi-Symbol Comparison (Normalized Returns)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Return (%)')
    ax.set_xlabel('Date')
    ax.legend()
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 8. Parameter Sensitivity Analysis

Test different parameter combinations to understand sensitivity

In [ ]:
# Parameter sweep configuration
RUN_PARAMETER_SWEEP = False  # Set to True to run

SWEEP_CONFIG = {
    'stop_loss_pct': [0.01, 0.02, 0.03],
    'take_profit_pct': [0.03, 0.04, 0.06],
}

if RUN_PARAMETER_SWEEP:
    sweep_results = []
    
    for sl in SWEEP_CONFIG['stop_loss_pct']:
        for tp in SWEEP_CONFIG['take_profit_pct']:
            print(f"Testing SL={sl*100:.1f}%, TP={tp*100:.1f}%...")
            try:
                res = run_ml_backtest(
                    symbol=SYMBOL,
                    start_date=START_DATE,
                    end_date=END_DATE,
                    timeframe=TIMEFRAME,
                    initial_capital=INITIAL_CAPITAL,
                    stop_loss_pct=sl,
                    take_profit_pct=tp,
                    save_trades=False,
                    save_equity_curve=False,
                )
                sweep_results.append({
                    'Stop Loss': f"{sl*100:.1f}%",
                    'Take Profit': f"{tp*100:.1f}%",
                    'Return': res.total_return_pct,
                    'Sharpe': res.sharpe_ratio,
                    'Win Rate': res.win_rate,
                    'Max DD': res.max_drawdown_pct,
                    'Trades': res.total_trades,
                })
            except Exception as e:
                print(f"  Error: {e}")
    
    # Display results
    sweep_df = pd.DataFrame(sweep_results)
    sweep_df = sweep_df.sort_values('Sharpe', ascending=False)
    print("\nPARAMETER SWEEP RESULTS (sorted by Sharpe)")
    print("=" * 80)
    display(sweep_df.style.format({
        'Return': '{:.2%}',
        'Sharpe': '{:.2f}',
        'Win Rate': '{:.1%}',
        'Max DD': '{:.2%}',
    }))
else:
    print("Parameter sweep disabled.")
    print("Set RUN_PARAMETER_SWEEP = True to run sensitivity analysis.")

## 9. Export Results

Export the backtest results for further analysis

In [ ]:
# Export options
EXPORT_DIR = Path('../runs/backtests')
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Export trades to CSV
if len(trades_df) > 0:
    trades_csv_path = EXPORT_DIR / f"{TAG}_trades.csv"
    trades_df.to_csv(trades_csv_path, index=False)
    print(f"Trades exported to: {trades_csv_path}")

# Export equity curve to CSV
if results.equity_curve:
    equity_csv_path = EXPORT_DIR / f"{TAG}_equity.csv"
    equity_df.to_csv(equity_csv_path)
    print(f"Equity curve exported to: {equity_csv_path}")

# Export metrics summary
metrics_path = EXPORT_DIR / f"{TAG}_metrics.csv"
metrics_df.to_csv(metrics_path, index=False)
print(f"Metrics exported to: {metrics_path}")

## 10. Summary

Final backtest summary and notes

In [ ]:
# Print final summary
print("\n" + "=" * 60)
print("BACKTEST SUMMARY")
print("=" * 60)
print(f"\nSymbol:          {SYMBOL}")
print(f"Period:          {START_DATE} to {END_DATE}")
print(f"Timeframe:       {TIMEFRAME}")
print(f"Initial Capital: ${INITIAL_CAPITAL:,.0f}")
print(f"Final Capital:   ${results.final_capital:,.2f}")
print(f"\nTotal Return:    {results.total_return_pct*100:+.2f}%")
print(f"Sharpe Ratio:    {results.sharpe_ratio:.2f}")
print(f"Max Drawdown:    {results.max_drawdown_pct*100:.2f}%")
print(f"Win Rate:        {results.win_rate*100:.1f}%")
print(f"Total Trades:    {results.total_trades}")
print("\n" + "=" * 60)

# Performance assessment
if results.sharpe_ratio > 1.5:
    assessment = "EXCELLENT - Strong risk-adjusted returns"
elif results.sharpe_ratio > 1.0:
    assessment = "GOOD - Above average risk-adjusted returns"
elif results.sharpe_ratio > 0.5:
    assessment = "MODERATE - Acceptable but room for improvement"
else:
    assessment = "NEEDS WORK - Consider parameter optimization"

print(f"Assessment: {assessment}")
print("=" * 60)

---

## Notes

### Data Limitations
- Historical backtesting uses **synthetic engine signals** derived from price/volume data
- Real options flow and dealer positioning data would improve accuracy
- Past performance does not guarantee future results

### Recommendations
1. Test on multiple symbols and timeframes
2. Use walk-forward optimization to avoid overfitting
3. Consider transaction costs and slippage in real trading
4. Paper trade before going live

### Resources
- [Alpaca Documentation](https://alpaca.markets/docs/)
- [GNOSIS System Architecture](../docs/architecture.md)
- [Engine Documentation](../docs/engines.md)